In [1]:
import numpy as np
import random as rng
from functools import partial
from time import time
import copy
from funs import *

In [3]:
w = 3
k = 10
alfa = 0.5
Theta = np.array([[0.375, 0.1, 0.14285714285714285],
                  [0.125, 0.2, 0.2857142857142857],
                  [0.25, 0.3, 0.14285714285714285],
                  [0.25, 0.4, 0.42857142857142855]], )
ThetaB = np.array([0.25, 0.25, 0.25, 0.25])

In [4]:
X = generate_data(Theta, ThetaB, alfa, k)

In [17]:
def run_em(data, alpha: int = None, max_iter: int = 500, min_diff: float = 1e-32) -> dict:
    tic = time()
    data = np.array(data, dtype='int32')
    w = data.shape[1]
    thetaB = np.full((4,), 1 / 4)
    theta = np.full((4, w), 1) / 4
    rep = 0
    theta_next = copy.deepcopy(theta)
    thetaB_next = copy.deepcopy(thetaB)
    norm = min_diff + 1
    normB = min_diff + 1

    results = dict({"data": data, "max_iter": max_iter, "min_diff": min_diff,
                    "alpha": alpha, "Theta": theta, "ThetaB": thetaB,
                    "last_diff": None, "last_diffB": None, "iterations": 0, "time_spent": None})

    while (rep < max_iter) & ((min_diff < norm) | (min_diff < normB)):
        # Calculating pi
        calc_pi_mapfunc = partial(calc_pii, theta=theta, thetaB=thetaB, alpha=alpha)
        pi = np.array(list(map(calc_pi_mapfunc, data)))
        for m in np.arange(4):
            # Derivative for Theta
            numerator = (np.array((data == m + 1), dtype='float32').T * pi).T
            theta_next[m, :] = np.apply_along_axis(np.sum, axis=0, arr=numerator) / np.sum(pi)
            # Derivative for ThetaB
            Al = np.apply_along_axis(np.sum, axis=1, arr=(data == m + 1))
            thetaB_next[m] = (np.sum((1 - pi) * Al)) / (w * np.sum(1 - pi))
        diff = theta_next - theta
        norm = ((diff ** 2).sum()) / 4 * w
        diffB = thetaB_next - thetaB
        normB = ((diffB ** 2).sum()) / 4
        theta = copy.deepcopy(theta_next)
        thetaB = copy.deepcopy(thetaB_next)
        rep += 1
    toc = time()

    results['Theta'] = theta
    results['ThetaB'] = thetaB
    results['alpha'] = alpha
    results['last_diff'] = norm
    results['last_diffB'] = normB
    results['iterations'] = rep
    results['time_spent'] = toc - tic

    return results

In [18]:
res = run_em(X, alpha=0.5)

In [20]:
res['iterations']

30

In [6]:
calc_dtv(res['Theta'], Theta)

1.5016002067701886